In [ ]:
import ftplib
from ftplib import FTP
import numpy as np
import os
import sys
sys.path.append('src')
import satpos
import RINEX
import gzip

In [ ]:
#获取BDS官方广播星历
year=2025
doy=50
save=1

#创建FTP对象
ftp=FTP()
ftp.encoding='utf-8'
ip="ftp2.csno-tarc.cn"
port=""
ftp.connect(ip)
ftp.login('pub','tarc')

f_name="tarc{:03d}0.{:02d}b".format(doy,year-2000)

#ftp.dir()
ftp.cwd('brdc')
ftp.cwd(str(year))
with open(f_name,"wb") as f:
    try:
        ftp.retrbinary("RETR {}".format(f_name),f.write,104857600)
        print("下载",f_name.format(doy,year-2000)," 成功")
    except:
        print("下载",f_name.format(doy,year-2000)," 失败")
ftp.quit()

if(not save):
    os.remove(f_name)

220-FileZilla Server 1.9.4
220-Please visit https://filezilla-project.org/
220 Welcome to data center of Test and Assessment Research Center of CSNO.
下载 tarc0500.25b  成功


In [31]:
#获取WUM精密星历产品
def get_WUM_SP3(year,doy,save,path='',product="WUM0MGXFIN",f_name="",innertial=300,return_=0,out_path=""):
    # 输入: 年, 年积日, 保存标志, 产品路径, 产品名称, 产品文件名, 星历间隔, 返回标志
    # 输出: 无返回值  /  Easy4PPP自有格式的精密星历SP3字典数组
    # 时间转换
    ut=satpos.epoch2time(satpos.COMMTIME(year,1,1,0,0,0))+(doy-1)*60*60*24
    gweek,gsec=satpos.time2gpst(ut)
    download_failed=0
    #print(gweek,gsec)
    
    #无f_name参数指定, 则默认长命名格式的WUM精密星历最终产品
    #若f_name参数指定, 则除save, path, f_name, return_外其余参数全部无效
    if(f_name==""):
        if(out_path!=""):
            os.makedirs(out_path,exist_ok=True)
        p_name="{}_{:04d}{:03d}0000_01D_{:02d}M_ORB.SP3.gz".format(product,year,doy,int(innertial/60))
        if(out_path!=""):
            f_name="{}/{}_{:04d}{:03d}0000_01D_{:02d}M_ORB.SP3.gz".format(out_path,product,year,doy,int(innertial/60))
        else:
            f_name="{}_{:04d}{:03d}0000_01D_{:02d}M_ORB.SP3.gz".format(product,year,doy,int(innertial/60))
    #创建FTP对象
    ftp=FTP()
    ftp.encoding='utf-8'
    ip="igs.gnsswhu.cn"
    ftp.connect(ip)
    ftp.login()
    #print(ftp.getwelcome())

    if(path==''):
        ftp.cwd('pub')
        ftp.cwd('gnss')
        ftp.cwd('products')
        ftp.cwd('mgex')
        ftp.cwd('{:04d}'.format(gweek))
    else:
        ftp.cwd(path)
    
    with open(f_name,"wb") as f:
        try:
            ftp.retrbinary("RETR {}".format(p_name),f.write,104857600)
            print("下载"," {}".format(f_name)," 成功")
        except:
            print("下载"," {}".format(f_name)," 失败")
            download_failed=1
    #断开FTP连接
    ftp.quit()
    if(download_failed):
        os.remove(f_name)
        return None

    #解压写入(仅支持gz文件)
    try:
        with gzip.open(f_name,'rb') as f_in:
            with open(f_name.replace(f_name.split(".")[-1],""),'wb') as f_out:
                f_out.write(f_in.read())
        os.remove(f_name)#解压完成删除压缩文件
    except:
        os.remove(f_name.replace(f_name.split(".")[-1],""))#解压完成删除压缩文件
        print("非 .gz 文件, 跳过自动解压")
        return None #非.gz文件, 返回
    
    if(return_):
        sp3=RINEX.getsp3(f_name.replace(f_name.split(".")[-1],""))
    
    if(not save):
        os.remove(f_name.replace(f_name.split(".")[-1],""))
    
    if(return_):
        return sp3
    else:
        return None

get_WUM_SP3(2025,35,save=1,innertial=300,out_path="data/NAVdata/Peph/20250350")

下载  data/NAVdata/Peph/20250350/WUM0MGXFIN_20250350000_01D_05M_ORB.SP3.gz  成功


In [45]:
#获取WUM精密钟差产品
def get_WUM_CLK(year,doy,save,path='',product="WUM0MGXFIN",f_name="",innertial=30,return_=0,out_path=""):
    # 输入: 年, 年积日, 保存标志, 产品路径, 产品名称, 产品文件名, 星历间隔, 返回标志
    # 输出: 无返回值  /  Easy4PPP自有格式的精密星历SP3字典数组
    # 时间转换
    ut=satpos.epoch2time(satpos.COMMTIME(year,1,1,0,0,0))+(doy-1)*60*60*24
    gweek,gsec=satpos.time2gpst(ut)
    download_failed=0
    #print(gweek,gsec)
    
    #无f_name参数指定, 则默认长命名格式的WUM精密星历最终产品
    #若f_name参数指定, 则除save, path, f_name, return_外其余参数全部无效
    if(f_name==""):
        if(out_path!=""):
            os.makedirs(out_path,exist_ok=True)
        p_name="{}_{:04d}{:03d}0000_01D_{:02d}S_CLK.CLK.gz".format(product,year,doy,int(innertial))
        if(out_path!=""):
            f_name="{}/{}_{:04d}{:03d}0000_01D_{:02d}S_CLK.CLK.gz".format(out_path,product,year,doy,int(innertial))
        else:
            f_name="{}_{:04d}{:03d}0000_01D_{:02d}S_CLK.CLK.gz".format(product,year,doy,int(innertial))
    #创建FTP对象
    ftp=FTP()
    ftp.encoding='utf-8'
    ip="igs.gnsswhu.cn"
    ftp.connect(ip)
    ftp.login()
    #print(ftp.getwelcome())

    if(path==''):
        ftp.cwd('pub')
        ftp.cwd('gnss')
        ftp.cwd('products')
        ftp.cwd('mgex')
        ftp.cwd('{:04d}'.format(gweek))
    else:
        ftp.cwd(path)
    
    with open(f_name,"wb") as f:
        try:
            ftp.retrbinary("RETR {}".format(p_name),f.write,104857600)
            print("下载"," {}".format(f_name)," 成功")
        except:
            print("下载"," {}".format(f_name)," 失败")
            download_failed=1
    #断开FTP连接
    ftp.quit()
    if(download_failed):
        os.remove(f_name)
        return None

    #解压写入(仅支持gz文件)
    try:
        with gzip.open(f_name,'rb') as f_in:
            with open(f_name.replace(f_name.split(".")[-1],""),'wb') as f_out:
                f_out.write(f_in.read())
        os.remove(f_name)#解压完成删除压缩文件
    except:
        os.remove(f_name.replace(f_name.split(".")[-1],""))#解压完成删除压缩文件
        print("非 .gz 文件, 跳过自动解压")
        return None #非.gz文件, 返回
    
    if(return_):
        sp3=RINEX.getclk(f_name.replace(f_name.split(".")[-1],""))
    
    if(not save):
        os.remove(f_name.replace(f_name.split(".")[-1],""))
    
    if(return_):
        return sp3
    else:
        return None

#get_WUM_CLK(2025,35,save=1,innertial=30,out_path="")

In [ ]:
#自动下载并合并精密星历与钟差
year=2025
doy=50
def autodownload_CLK_SP3(year,doy,save=0,out_path=""):
    sp31=get_WUM_SP3(year,doy-1,save=save,innertial=300,return_=1,out_path=out_path)
    sp32=get_WUM_SP3(year,doy,save=save,innertial=300,return_=1,out_path=out_path)
    sp33=get_WUM_SP3(year,doy+1,save=save,innertial=300,return_=1,out_path=out_path)

    clk1=get_WUM_CLK(year,doy-1,save=save,innertial=30,return_=1,out_path=out_path)
    clk2=get_WUM_CLK(year,doy,save=save,innertial=30,return_=1,out_path=out_path)
    clk3=get_WUM_CLK(year,doy+1,save=save,innertial=30,return_=1,out_path=out_path)
    
    return list(np.concatenate([np.array(sp31),np.array(sp32),np.array(sp33)])),list(np.concatenate([np.array(clk1),np.array(clk2),np.array(clk3)]))

IGS,CLK=autodownload_CLK_SP3(year,doy)

下载  WUM0MGXFIN_20250490000_01D_05M_ORB.SP3.gz  成功
下载  WUM0MGXFIN_20250500000_01D_05M_ORB.SP3.gz  成功
下载  WUM0MGXFIN_20250510000_01D_05M_ORB.SP3.gz  成功
下载  WUM0MGXFIN_20250490000_01D_30S_CLK.CLK.gz  成功
下载  WUM0MGXFIN_20250500000_01D_30S_CLK.CLK.gz  成功
下载  WUM0MGXFIN_20250510000_01D_30S_CLK.CLK.gz  成功
